# Предобработка второй бд

In [58]:
import pandas as pd
import numpy as np

data2 = pd.read_excel('data/Database_2.xlsx')

### Отбрасываем признаки

In [59]:
# drop_cols = [
#     'Animal?', 'Test indicator', 'Biochemical metric', 
#     'Publication year', 'Particle ID', 'Reference DOI', 
#     'Interference checked (Y/N)', 'Colloidal stability checked (Y/N)', 'Positive control (Y/N)',
#     'Cell age: embryonic (E), Adult (A)', 'Cell-organ/tissue source'
# ]
# data2 = data2.drop(columns = drop_cols)

### Переименовываем признаки

In [60]:
data2.rename(
    columns = {
        'Nanoparticle': 'material_type', 
        'Concentration μM': 'concentration_ug/ml',
        '% Cell viability' : 'viability', 
        'coat': 'coating', 
        'Diameter (nm)': 'size_nm',
        'Zeta potential (mV)': 'surface_charge_mv', 
        'Cells': 'cell_type', 
        'Cell line (L)/primary cells (P)': 'cell_line', 
        'Human(H)/Animal(A) cells': 'animal',
        'Cell morphology': 'cell_morphology', 
        'Exposure time (h)': 'time_h', 
        'Test': 'test', 
        'Type: Organic (O)/inorganic (I)': 'type',
        'Animal?': 'animal',
        'Test indicator': 'test_indicator', 
        'Biochemical metric': 'biochemical_metrics', 
        'Publication year': 'publication_year', 
        'Particle ID': 'particle_id', 
        'Reference DOI': 'reference_doi', 
        'Interference checked (Y/N)': 'interference_test', 
        'Colloidal stability checked (Y/N)': 'colloidal_stability', 
        'Positive control (Y/N)': 'positive_control',
        'Cell age: embryonic (E), Adult (A)': 'cell_age', 
        'Cell-organ/tissue source': 'cell_organ',
        }, 
        inplace = True)

### Переименуем соединения

In [61]:
mt_dict = {
    'Copper Oxide': 'CuO',
    'Zinc oxide': 'ZnO', 
    'Iron oxide': 'Fe2O3', 
    'Hydroxyapatite': 'Ca10(PO4)6(OH)2', 
    'Polystyrene': '(C8H8)n'
    }
data2['material_type'].replace(mt_dict, inplace = True)

c:\Users\Denis\anaconda3\lib\site-packages\pandas\core\series.py:4509: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


### Заполним пропуски

In [62]:
data2.loc[21, 'cell_morphology'] = 'Endothelial'
data2.loc[98, 'cell_age'] = 'A'
data2.loc[31, 'test'] = 'MTT'
data2.loc[106, 'test_indicator'] = 'toluylene red'
data2.loc[69, 'biochemical_metrics'] = 'cell membrane integrity'

### Сохраняем

In [63]:
data2.to_excel('data/data2_prep.xlsx', index=False)